In [19]:
# %pip install -q evaluate Pillow==9.0.0
# %pip install -q transformers pytesseract datasets rouge-score nltk py7zr --upgrade
# %pip install -q numpy pandas scikit-learn

In [7]:
import pandas as pd

In [28]:
df = pd.read_csv("../data/train_output.csv")
df ["output"] = df["entity_name"] + "\n" + df["output"]
df = df[["entity_value", "output"]].dropna()
df.head()
X, y = df["output"].tolist(), df["entity_value"].tolist()


/tmp/ipykernel_226762/333120130.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train_output.csv")


In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/home/consularparadi/Coding/Dev/conda/envs/ml/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [30]:
from datasets import concatenate_datasets

tokenized_inputs = tokenizer(X, truncation=True)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

tokenized_targets = tokenizer(y, truncation=True)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Max source length: 512
Max target length: 11


In [40]:
def preprocess_function(sample, padding='max_length'):
    inputs = sample["input"]
    targets = sample["target"]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == 'max_length':
        labels["input"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input"]
    return model_inputs

dataset = pd.DataFrame({"input": X, "target": y})
for idx, row in dataset.iterrows():
    print(preprocess_function(row))
    # dataset.at[idx, "input"] = model_inputs["input_ids"]
    # dataset.at[idx, "target"] = model_inputs["labels"]

TypeError: 'int' object is not iterable

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)